In [14]:
import re
from itertools import combinations

In [20]:
def otkryvalka_fayla (put_k_faylu):
    """ Получаем на вход путь к исходному файлу
    считываем файл в строку
    возвращаем строку
    """
    with open (put_k_faylu, 'r') as openfile:
        fayl_kak_stroka = openfile.read()
        return fayl_kak_stroka

In [27]:
def vydelyalka_obyektov (kakoy_to_text):
    """ Получаем на вход текст
    Возвращаем список интересующих нас в этом тексте объектов
    (как список строк) -- это будущие узлы нашего графа
    """
    obyekty = re.findall (r'lemma = \"[а-я]+\"',kakoy_to_text) # намеренно примитивная логика
    # как раз здесь вы можете встроить что угодно более сложное
    return obyekty

In [21]:
def vydelyalka_svyazey (abzac):
    """ Получаем на вход абзац как строку
    выделяем в нем все интересующие нас объекты с помощью vydelyalka_obyektov
    создаем список пар "все со всеми" при помощи combinations
    возвращаем спиоск пар
    """
    obyekty_abzaca = vydelyalka_obyektov (abzac)
    obyekty_abzaca.sort() # отсортируем, чтобы нам не мешал разный порядок пар
    spisok_svyazey_abzaca = []
    tekushaya_para = []
    spisok_svyazey_abzaca = list(combinations(obyekty_abzaca, 2))
    return spisok_svyazey_abzaca

In [22]:
def obhodchik_abzacev (text_s_mnojestvom_abzacev):
    """ Получаем на вход файл как строку
    превращаем его в список абзацев
    обходим абзацы
    вытаскиваем из каждого его связи с помощью vydelyalka_svyazey
    кладем все связи всех абзацев в один список
    возвращаем список
    """
    spisok_abzacev = text_s_mnojestvom_abzacev.split('\n')
    spisok_vseh_svyazey = []
    for abzac in spisok_abzacev:
        print ('Текст абзаца: ', abzac)
        svyazi_tekushego_abzaca = vydelyalka_svyazey (abzac)
        print ('Все связи абзаца: ', svyazi_tekushego_abzaca)
        spisok_vseh_svyazey.extend (svyazi_tekushego_abzaca)
    print ('Все связи в документе: ', spisok_vseh_svyazey)    
    return spisok_vseh_svyazey

In [23]:
def zapis_v_graf (spisok_vseh_svyazey):
    """ Получаем на вход список связей в виде пар
    превращаем его в множество пар (убиваем дубликаты)
    возвращаем множество пар
    """
    graf = set (spisok_vseh_svyazey)
    return graf

In [24]:
def zapis_v_graf_s_vesami (spisok_vseh_svyazey):
    """ Получаем на вход список связей в виде пар
    превращаем его в словарь пар
    ключ -- пара, значение -- сколько раз такая пара встретилась(это будет вес связи в графе)
    возвращаем множество пар
    """
    graf = {}
    for svyaz in spisok_vseh_svyazey:
        if svyaz in graf:
            graf [svyaz] += 1
        else:
            graf [svyaz] = 1
    return graf

In [25]:
def zapis_v_fayl (put_k_vyhodnomu, graf):
    """ Получаем на вход путь к выходному файлу
    и граф в виде
    -- либо множества (случай невзвешеннго графа)
    -- либо словаря (случай взвешеннго графа)
    пишем его в файл построчно: одна связь (пара, ребро) -- одна строка
    """
    with open (put_k_vyhodnomu,'w') as output_file:
        for rebro in graf:
            if type(graf) == set: # случай невзвешенного графа, полученного функцией zapis_v_graf
                output_file.write (rebro[0]+','+rebro[1]+'\n')
            elif type(graf) == dict: # случай взвешенного графа, полученного функцией zapis_v_graf_s_vesami
                ves_rebra = graf [rebro]
                output_file.write (rebro[0]+','+rebro[1]+ ',' + str(ves_rebra) + '\n')

In [ ]:
if __name__ == "__main__":
    while (True):
        vhodn_fayl = input ('путь к файлу с текстом: ')
        vyhodn_fayl = input ('куда записать: ')
        otkrytiy_fayl = otkryvalka_fayla (vhodn_fayl)
        spisok_svyazey = obhodchik_abzacev(otkrytiy_fayl)
        mnojestvo_svyazey_s_vesami = zapis_v_graf_s_vesami (spisok_svyazey)
        zapis_v_fayl (vyhodn_fayl, mnojestvo_svyazey_s_vesami)

путь к файлу с текстом: /Users/tonleon/Documents/GitHub/Coursework-2020/Marked_text.txt
куда записать: /Users/tonleon/Documents/GitHub/Coursework-2020/Grafs.txt
Текст абзаца:    [Из записей Веры Николаевны:]
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     1 янв. 1928/19 дек. 1927.
Все связи абзаца:  []
Текст абзаца:     Францию с Новым Годом! А я еще не чувствую, что Новый год идет, хотя уже 8 лет встречаю его во Франции, и все еще не привыкла. 
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     2 января.
Все связи абзаца:  []
Текст абзаца:      В "Днях" напечатан рассказ Г. Н. <persName ref="#kuznetsova" lemma = "кузнецова" #>[Кузнецовой.</persName> -- М. Г.]. Статья <persName ref="#makeev" lemma = "макеев" #>Макеева</persName> о "Совр. Зап." Восхищается "Божьим Древом"1, но говорит, что эта жизнь канула в вечность.
Все связи абзаца:  [('lemma = "кузнецова"', 'lemma = "макеев"')]
Текст абзаца:     Ян сказал:  Я пишу о душе р

Текст абзаца:     Завтракали в кабинете с Яном. Я всегда испытываю счастье, когда он после болезни начинает жить нормальной жизнью. Особенно остро я чувствовала это на Суэцком канале в 1910 году на Рождестве 
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     11 мая.
Все связи абзаца:  []
Текст абзаца:     Из письма <persName ref="#kartashev" lemma = "карташев" #>Карташева</persName> (открытка): "Пасха была радостная. <persName ref="#kedrov" lemma = "кедров" #>Кедровы</persName> вернулись из Америки в вел. Пятницу и украсили нашу заутреню свои пением. Было хорошо, полно, все устроилось густо по-русски: красные свечки, колокола, масса народу. Нет выше нашей богослужебной красоты, особенно, если знать богослужение".
Все связи абзаца:  [('lemma = "карташев"', 'lemma = "кедров"')]
Текст абзаца:     Всегда, при всяком общении с <persName ref="#kartashev" lemma = "карташев" #>Карташевым</persName> я испытываю трудно впечатление чего-то настоящего, большого.  О че

Все связи абзаца:  []
Текст абзаца:     31 июля.
Все связи абзаца:  []
Текст абзаца:      Ян в лучшем настроении. Во время прогулки он вел с нами разговор на литературные темы. Он только обескураживает Лося насчет исторических романов, -- до чего это бунинская черта, кто близко -- тот ничего не может делать.  А на Леню это подействовало. А на самого Яна как действовали братья уверяя, что он ничего не знает и ни о чем писать не может. А между тем, Юлий Ал. считал его очень талантливым.
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     2 августа.
Все связи абзаца:  []
Текст абзаца:     <persName ref="#sorin" lemma = "сорин" #>Сорин</persName> был недолго, т. к. он ежедневно ездит в два часа на этюды.  Он рассказывал, что когда жил в Арзамасе, <persName ref="#gor'kij" lemma = "горький" #>Горький</persName> говорил ему: "Вот поезжайте с Катей в Нижний и купите ей шелку на платье, такого, чтобы шуршал, люблю шуршащие юбки".
Все связи абзаца:  [('lemma = "горьки

Все связи абзаца:  []
Текст абзаца:     2 января.
Все связи абзаца:  []
Текст абзаца:      Ян волнуется из-за норвежского издательства. Если бы дело это устроилось, то мы были бы на весь год спасены. 
Все связи абзаца:  []
Текст абзаца:     Думала поехать завтра в церковь, но рассчитала -- не хватит денег.  При бедности самое тяжелое -- праздники. 
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     3 января.
Все связи абзаца:  []
Текст абзаца:      Вырубов пополнел, стал блестящ по виду, все так же весел, неистощим.  Какая свобода!  какая легкость разговора, перескакивание с темы на тему, и все без задержки, без напряжения. 
Все связи абзаца:  []
Текст абзаца:     О <persName ref="#savinkov" lemma = "савинков" #>Савинкове:</persName> Он случайно встретился с ним накануне отъезда его в Россию  Савинков был выпивши и стал звать его зайти куда-нибудь. Вблизи оказался ресторан. 
Все связи абзаца:  []
Текст абзаца:     -- Савинков, как вы знаете, -- рассказывает

Все связи абзаца:  []
Текст абзаца:     Кламар, 1933, 5 июня К. <persName ref="#bal'mont" lemma = "бальмонт" #>Бальмонт.]
Все связи абзаца:  []
Текст абзаца:  </persName>  
Все связи абзаца:  []
Текст абзаца:     Я написал в ответ:
Все связи абзаца:  []
Текст абзаца:     Милый! Пусть мы только псы
Все связи абзаца:  []
Текст абзаца:     Все равно: как много шавок,
Все связи абзаца:  []
Текст абзаца:     У которых только навык
Все связи абзаца:  []
Текст абзаца:     Заменяет все красы.
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     [Из записей Веры Николаевны:]
Все связи абзаца:  []
Текст абзаца:    
Все связи абзаца:  []
Текст абзаца:     14 июля.
Все связи абзаца:  []
Текст абзаца:     Эти дни очень тяжело. Вести о Павлике неутешительные. Видимо, полного выздоровления не будет. Боюсь, начнется разложение личности. 
Все связи абзаца:  []
Текст абзаца:     Я кончила перестукивать книгу <persName ref="#kurdjumov" lemma = "курдюмов" #>Курдюмова.</persName>